In [2]:
import duckdb

In [40]:
# Crear/conectar a una base de datos en disco
con = duckdb.connect(database='../input/db/labo3.duckdb')  # Cambia la ruta si prefieres otro nombre o carpeta

# Crear tablas desde los CSV en duckdb
con.execute("""
    CREATE OR REPLACE TABLE sell_in AS 
    SELECT  
            CONCAT(periodo, '|', customer_id, '|', product_id) AS key_periodo_customer_producto,
            CAST(STRPTIME(CAST(periodo AS VARCHAR) || '01', '%Y%m%d') AS DATE) AS periodo_fecha,
            periodo,
            customer_id,
            product_id,
            plan_precios_cuidados,
            cust_request_qty,
            cust_request_tn,
            tn
    FROM read_csv(
        '../input/sell-in.txt', 
        delim='\t',
        columns={
            'periodo': 'INTEGER',
            'customer_id': 'INTEGER',
            'product_id': 'INTEGER',
            'plan_precios_cuidados': 'BOOLEAN', 
            'cust_request_qty': 'DECIMAL(6,2)',
            'cust_request_tn': 'DECIMAL(8,5)',
            'tn': 'DECIMAL(8,5)'
        }
    )
""")

con.execute("""
    CREATE OR REPLACE TABLE tb_productos AS 
    SELECT * FROM read_csv_auto('../input/tb_productos.txt', sep='\t');
""")
con.execute("""
    CREATE OR REPLACE TABLE tb_productos_a_predecir AS 
    SELECT * FROM read_csv_auto('../input/tb_productos_a_predecir.txt', sep='\t');
""")
con.execute("""
    CREATE OR REPLACE TABLE tb_stocks AS 
    SELECT *,
            CONCAT(periodo, '|', product_id) AS key_periodo_producto
            FROM read_csv_auto('../input/tb_stocks.txt', sep='\t');
""")

con.close()

#### Se validan datos levantados de los txt.
 - Control de totales en productos, ventas y productos a predecir.
 - Se eliminan de la tabla de ventas los productos que no no están en la tabla productos.
 - Se elimina la tabla de ventas original.

In [41]:
# Conectar a la base existente
con = duckdb.connect(database='../input/db/labo3.duckdb')

# Cantidad de productos de sell_in que no están en tb_productos
query = """
SELECT distinct product_id FROM sell_in
where product_id not in (select product_id from tb_productos)
"""
cantidad = con.execute(query).fetchdf()
print(f"Cantidad de productos de sell_in que no están en tb_productos: {len(cantidad)}")
#display(cantidad)


### Validad si algunos de los productos no están en la tabla tb_productos los tengo que predecir.
ids = cantidad['product_id'].tolist()
query = f"""
SELECT product_id 
FROM tb_productos_a_predecir
WHERE product_id IN ({','.join(map(str, ids))})
"""
en_a_predecir = con.execute(query).fetchdf()
print(f"Cantidad de esos productos que están en tb_productos_a_predecir: {len(en_a_predecir)}")
#display(en_a_predecir)

## VALIDAR PRODUCTOS DUPLICADOS EN DF_PRODUCTOS
query = """
SELECT product_id, count(product_id) as cantidad FROM tb_productos
GROUP BY product_id
HAVING count(product_id) > 1
"""
duplicados = con.execute(query).fetchdf()
print(f"Cantidad de productos duplicados en tb_productos: {len(duplicados)}")  
#display(duplicados)


# Generar una nueva tabla sell_in_clean con los productos que están en tb_productos, los demas se eliminan.
con.execute("""
CREATE OR REPLACE TABLE sell_in_clean AS
SELECT sell_in.*
FROM sell_in
JOIN tb_productos ON sell_in.product_id = tb_productos.product_id
""")
con.execute("""DROP TABLE IF EXISTS sell_in""")

## Cuento registros luego de limpiar los product_id de sell_in que no estan en tb_productos
query = """
SELECT count(product_id) as cantidad FROM sell_in_clean
GROUP BY product_id
"""
cantidad_clean = con.execute(query).fetchdf()
print(f"Cantidad de productos en sell_in_clean: {len(cantidad_clean)}")

query = """SELECT DISTINCT product_id FROM sell_in_clean p"""
cantidad_productos = con.execute(query).fetchdf()
print(f"Cantidad de productos distintos en sell_in_clean: {len(cantidad_productos)}")  

query = """SELECT DISTINCT customer_id FROM sell_in_clean"""
cantidad_clientes = con.execute(query).fetchdf()
print(f"Cantidad de clientes distintos en sell_in_clean: {len(cantidad_clientes)}") 

query = """SELECT DISTINCT periodo from sell_in_clean"""
cantidad_periodos = con.execute(query).fetchdf()
print(f"Cantidad de periodos distintos en sell_in_clean: {len(cantidad_periodos)}")

#multiplico cantidad_clientes * cantidad_productos * cantidad_periodos
total_combinaciones = len(cantidad_clientes) * len(cantidad_productos) * len(cantidad_periodos)
print(f"Total de combinaciones posibles (clientes * productos * periodos): {total_combinaciones}")


con.close()


Cantidad de productos de sell_in que no están en tb_productos: 45
Cantidad de esos productos que están en tb_productos_a_predecir: 0
Cantidad de productos duplicados en tb_productos: 0
Cantidad de productos en sell_in_clean: 1188
Cantidad de productos distintos en sell_in_clean: 1188
Cantidad de clientes distintos en sell_in_clean: 597
Cantidad de periodos distintos en sell_in_clean: 36
Total de combinaciones posibles (clientes * productos * periodos): 25532496


#### Determinacion de fechas por producto / cliente, total de combinaciones por cliente * producto * periodo
 - Se limpian los registros luego del producto cartesiano
 - Se completan con 0 los datos sinteticos
 - Se calcula antiguedad de productos y clientes.


In [ ]:
con = duckdb.connect(database='../input/db/labo3.duckdb')

# creo una tabla temporal con la primer y última venta de cada producto
con.execute("""
CREATE OR REPLACE TABLE tmp_productos_primer_ultima_venta AS
SELECT product_id, 
    MIN(periodo_fecha) AS primer_venta_producto, 
    MAX(periodo_fecha) AS ultima_venta_producto
FROM sell_in_clean
GROUP BY product_id
""")

# Query to display the data from the table
#query = """SELECT * FROM tmp_productos_primer_ultima_venta"""
#productos_primer_ultima_venta = con.execute(query).fetchdf()
#display(productos_primer_ultima_venta.head())
#display(productos_primer_ultima_venta.shape)


con.execute("""
CREATE OR REPLACE TABLE tmp_clientes_primer_ultima_venta AS            
SELECT customer_id,
       min(periodo_fecha) AS primer_venta_cliente,
       max(periodo_fecha) AS ultima_venta_cliente
FROM sell_in_clean
GROUP BY customer_id
"""
)
#query = """SELECT * FROM tmp_clientes_primer_ultima_venta"""
#clientes_primer_ultima_venta = con.execute(query).fetchdf()      
#display(clientes_primer_ultima_venta.head())
#display(productos_primer_ultima_venta.shape)



#MEGA JOIN productos, clientes y periodo
con.execute("""
CREATE OR REPLACE TABLE cross_join_pcp AS
SELECT p.product_id, 
            c.customer_id,
            pe.periodo,
            CAST(STRPTIME(CAST(periodo AS VARCHAR) || '01', '%Y%m%d') AS DATE) AS periodo_fecha,
            CONCAT(pe.periodo, '|', c.customer_id, '|', p.product_id) AS key_periodo_customer_producto
FROM (SELECT DISTINCT product_id FROM sell_in_clean) p
CROSS JOIN (SELECT DISTINCT customer_id FROM sell_in_clean) c
CROSS JOIN (SELECT DISTINCT periodo FROM sell_in_clean) pe
""")
#query = """sELECT * FROM cross_join_pcp"""
#mega_join = con.execute(query).fetchdf()
#display(mega_join.shape)
#display(mega_join.head(5))

# obtengo la fecha de la primer venta de cada producto y los agrego a la tabla cross_join_pcp
con.execute("""
CREATE OR REPLACE TABLE tmp_cross_join_con_fechas AS
SELECT     
        cj.*, 
        pp.primer_venta_producto as PVP,
        pp.ultima_venta_producto as UVP,
        cc.primer_venta_cliente as PVC,
        cc.ultima_venta_cliente as UVC 
FROM cross_join_pcp cj
LEFT JOIN tmp_productos_primer_ultima_venta pp ON cj.product_id = pp.product_id
LEFT JOIN tmp_clientes_primer_ultima_venta cc ON cj.customer_id = cc.customer_id            
""")
#query = """SELECT * FROM tmp_cross_join_con_fechas"""
#mega_join_con_fechas = con.execute(query).fetchdf()
#display(mega_join_con_fechas.shape)
#display(mega_join_con_fechas.head(5))


#elimino la tabla temporal
con.execute("DROP TABLE IF EXISTS cross_join_pcp")

# Agrego a la tabla con fechas los filtros para luego agregar los ceros.
# la columna antiguedad la genero para llenarla luego, y la columna plan_precios_cuidados la genero para luego actualizarla
con.execute("""
CREATE OR REPLACE TABLE tmp_cross_join_filtrada AS
SELECT
    1 AS registro_sintetico,
    1 as antiguedad_cliente,
    1 as antiguedad_producto,
    '' as key_periodo_producto,
    '' as key_customer_producto_periodo,      
	ccf.*, 
  	si.plan_precios_cuidados,
  	si.cust_request_qty,
  	si.cust_request_tn,
  	si.tn
FROM tmp_cross_join_con_fechas ccf
LEFT JOIN sell_in_clean si ON ccf.key_periodo_customer_producto = si.key_periodo_customer_producto
where ccf.periodo_fecha >= pvp
and ccf.periodo_fecha >= pvc
and ccf.periodo_fecha <= uvp
""")
con.execute("DROP TABLE IF EXISTS tmp_cross_join_con_fechas")


## agrego las columnas periodo, customer_id y product_id a la tabla tmp_cross_join_filtrada
con.execute("""UPDATE tmp_cross_join_filtrada set 
registro_sintetico = 0,      
WHERE tn is not NULL
""")

#Agrego la antiguedad de cada cliente y cada producto, seteo todo precios cuidados en false, luego actualizo los que corresponden
con.execute("""UPDATE tmp_cross_join_filtrada set 
antiguedad_cliente = DATEDIFF('month', PVC, periodo_fecha),
antiguedad_producto = DATEDIFF('month', PVP, periodo_fecha),
plan_precios_cuidados = false,
key_periodo_producto = CONCAT(periodo, '|', product_id),
key_customer_producto_periodo = CONCAT(customer_id, '|', product_id, '|', periodo)
""")

# Hago cero los campos sinteticos para tn, cust_request_qty y cust_request_tn
con.execute("""UPDATE tmp_cross_join_filtrada set
tn = 0,
cust_request_qty = 0,
cust_request_tn = 0
WHERE registro_sintetico = 1
""")

# actualizo los campos sinteticos de precios cuidados, pongo en true lo que era true.
con.execute("""UPDATE tmp_cross_join_filtrada set
plan_precios_cuidados = true
WHERE product_id in (SELECT DISTINCT product_id from sell_in_clean WHERE plan_precios_cuidados = true)
""")

# agrego el stock de cada producto en cada periodo
con.execute("""cREATE OR REPLACE TABLE tmp_cross_join_filtrada AS
SELECT cjf.*,
       st.stock_final
FROM tmp_cross_join_filtrada cjf
LEFT JOIN tb_stocks st ON cjf.key_periodo_producto = st.key_periodo_producto
""")


#query = """SELECT * FROM tmp_cross_join_filtrada"""
#tmp_cross_join_filtrada = con.execute(query).fetchdf()
#display(tmp_cross_join_filtrada.shape)
#display(tmp_cross_join_filtrada.head(5))

con.close()